<a href="https://colab.research.google.com/github/firda22/AR-Rute-Kereta/blob/main/Rute_Tercepat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Flask pyngrok networkx matplotlib


In [2]:
import heapq
import networkx as nx
import matplotlib.pyplot as plt
from flask import Flask, render_template, request, Response
from pyngrok import ngrok, conf
import io
import base64

# Graph data (Graph rute kereta)
graph_data = {
    'BWI': {'KTG': 13, 'YK': 678, 'LPN': 777,'ML':  420, 'MLK': 436, 'PSE': 974, 'JNG': 960, 'SGU': 346,'SBI':  381,'KM':  759,'GB':  777, 'JR': 118},
    'KTG': {'BWI': 13,'YK':  693,'LPN':  793,'ML':  440, 'MLK': 456, 'PSE': 990,'JNG':  976, 'SGU': 361,'SBI':  396, 'KM': 774, 'GB': 792, 'JR': 134},
    'YK': {'BWI': 663,'KTG':  680,'LPN':  4, 'ML': 335, 'MLK': 327, 'PSE': 432, 'JNG': 366,'SGU':  231,'BD': 368,'KAC': 388, 'KM': 64, 'GB': 90, 'JR': 537},
    'LPN': {'BWI': 752, 'KTG': 770, 'YK': 4,  'PSE': 451,'JNG':  439,'SGU':  317,'KAC': 474, 'KM': 80, 'GB': 97,'JR':  601},
    'ML': {'BWI': 397, 'KTG': 414, 'YK': 327, 'MLK': 6,'PSE':  788,'JNG':  718, 'SGU': 111,'SBI':  130, 'BD': 738, 'KAC': 728,'KM': 402,'GB':  459,'JR':  258},
    'MLK': {'BWI': 413, 'KTG': 430,'YK':  326, 'ML': 6, 'PSE': 820, 'JNG': 750,'BD': 728,'KAC':  718,'KM': 396,'GB':  451,'JR':  274},
    'PSE': {'BWI': 985, 'KTG': 1000, 'YK': 407, 'LPN': 469, 'ML': 775, 'MLK': 878, 'JNG':  11,'SGU':  620, 'SBI': 597, 'BD': 199, 'KAC': 212, 'CMI': 186,'KM': 350,'GB':  332,'JR':  856},
    'JNG': {'BWI': 972, 'KTG': 987, 'YK': 352 , 'LPN': 454, 'ML': 945, 'MLK': 936, 'PSE': 10, 'SGU': 607, 'SBI': 584, 'BD': 155,'KAC':  198, 'CMI': 143,'KM':  357, 'GB': 339,'JR': 843},
    'SGU': {'BWI': 353, 'KTG': 370, 'YK': 222, 'LPN': 288, 'ML': 117, 'PSE': 602,'JNG':  581, 'SBI':  14, 'BD': 593, 'KAC': 650, 'KM': 301,'GB':  339,'JR':  215},
    'SBI': {'BWI': 380, 'KTG': 395, 'ML': 136,'PSE':  583,'JNG':  497, 'SGU': 14, 'BD': 680,'CMI': 668,'JR':  237},
    'BD': {'YK': 355, 'ML': 785, 'MLK': 776, 'PSE': 243,'JNG':  147,'SGU':  595,'SBI':  674, 'KAC': 9,'CMI':  10,'KM': 301, 'GB': 313},
    'KAC': {'YK': 395, 'LPN': 478, 'ML': 773,'MLK':  764, 'PSE': 229,'JNG':  217, 'SGU': 657, 'BD': 8, 'CMI': 25, 'KM': 320,'GB':  302},
    'CMI': {'PSE': 199, 'JNG': 135,'SBI': 660,'BD':  9, 'KAC': 23},
    'KM': {'BWI': 752, 'KTG': 769,'YK':  66,'LPN':  84, 'ML': 409, 'MLK': 401, 'PSE': 364, 'JNG': 300, 'SGU': 303, 'BD': 312,'KAC':  320, 'GB': 15,'JR':  626},
    'GB': {'BWI': 771, 'KTG': 788,'YK':  90, 'LPN': 103,'ML':  479, 'MLK': 471, 'PSE': 346,'JNG': 334, 'SGU': 350, 'BD': 322,'KAC':  309,'KM': 16, 'JR': 645},
    'JR': {'BWI': 115, 'KTG': 132, 'YK': 526, 'LPN': 605, 'ML': 284, 'MLK': 300, 'PSE':850,'JNG':  812, 'SGU': 216, 'SBI': 245,'KM':  613,'GB':  631}
}




# Fungsi Dijkstra
def dijkstra(graph, start, goal):
    priority_queue = []
    heapq.heappush(priority_queue, (0, start))
    distances = {node: float('inf') for node in graph}
    distances[start] = 0
    previous_nodes = {node: None for node in graph}

    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)

        if current_distance > distances[current_node]:
            continue

        for neighbor, weight in graph[current_node].items():
            distance = current_distance + weight

            if distance < distances[neighbor]:
                distances[neighbor] = distance
                previous_nodes[neighbor] = current_node
                heapq.heappush(priority_queue, (distance, neighbor))

    # Backtrack untuk menemukan path
    path, current_node = [], goal
    while current_node is not None:
        path.insert(0, current_node)
        current_node = previous_nodes[current_node]

    return path, distances[goal]

# Membuat grafik dengan networkx dan menggambar rute tercepat, dengan bobot pada edge
def create_graph_image(path, graph_data):
    # Membuat objek graf kosong
    G = nx.Graph()

    # Menambahkan node dan edge ke dalam grafik
    for node in graph_data:
        for neighbor, weight in graph_data[node].items():
            G.add_edge(node, neighbor, weight=weight)

    # Menentukan posisi node secara otomatis
    pos = nx.spring_layout(G)

    # Gambar graf
    plt.figure(figsize=(10, 8))
    nx.draw(G, pos, with_labels=True, node_size=2000, node_color='skyblue', font_size=10, font_weight='bold', edge_color='gray')

    # Menambahkan label bobot pada setiap edge
    edge_labels = nx.get_edge_attributes(G, 'weight')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)

    # Menyorot jalur dari hasil Dijkstra
    path_edges = [(path[i], path[i+1]) for i in range(len(path)-1)]
    nx.draw_networkx_edges(G, pos, edgelist=path_edges, edge_color='red', width=2)

    # Menyimpan gambar ke dalam memori (BytesIO)
    img_stream = io.BytesIO()
    plt.savefig(img_stream, format='png')
    img_stream.seek(0)  # Resetkan posisi pointer stream
    plt.close()  # Menutup gambar untuk mencegah tampilan ganda

    return img_stream

# Membuat instance Flask
app = Flask(__name__)

# Route untuk halaman utama
@app.route('/')
def home():
    return '''
    <h1>Cari Rute Kereta Api Tercepat</h1>
    <form action="/find_route" method="POST">
        <label for="start">Stasiun Asal:</label>
        <input type="text" id="start" name="start" required><br><br>
        <label for="goal">Stasiun Tujuan:</label>
        <input type="text" id="goal" name="goal" required><br><br>
        <button type="submit">Cari Rute</button>
    </form>
    '''

# Route untuk menerima form input dan menampilkan hasilnya
@app.route('/find_route', methods=['POST'])
def find_route():
    start = request.form['start']
    goal = request.form['goal']

    if start not in graph_data or goal not in graph_data:
        return "Stasiun tidak ditemukan!"

    # Mencari rute menggunakan Dijkstra
    path, cost = dijkstra(graph_data, start, goal)

    # Membuat gambar grafik dari rute tercepat
    img_stream = create_graph_image(path, graph_data)

    # Menampilkan hasil dan gambar langsung di halaman
    img_data = base64.b64encode(img_stream.getvalue()).decode('utf-8')
    return f'''
    <h2>Rute Tercepat dari {start} ke {goal}</h2>
    <p>Rute: {path}</p>
    <p>Lama perjalanan: {cost}</p>
    <br>
    <img src="data:image/png;base64,{img_data}" alt="Graf Rute Kereta" />
    <br>
    <a href="/">Kembali</a>
    '''

# Menjalankan aplikasi Flask
if __name__ == '__main__':
    # Mengatur authtoken ngrok
    conf.get_default().auth_token = "2psUgQ3VxXnopiZcIqEmp0x4oYX_4c6ZzerBikirL9kimL4bF"  # Ganti dengan authtoken Anda

    # Membuat tunnel ngrok untuk aplikasi Flask
    public_url = ngrok.connect(5000)
    print('Flask app is running on:', public_url)
    app.run(port=5000)


Flask app is running on: NgrokTunnel: "https://be12-34-125-220-216.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [07/Dec/2024 07:44:20] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Dec/2024 07:44:22] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [07/Dec/2024 07:44:30] "POST /find_route HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Dec/2024 07:44:53] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Dec/2024 07:45:13] "POST /find_route HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Dec/2024 07:45:23] "GET / HTTP/1.1" 200 -
